# Proyecto RI

### Dataset obtenido de Kaggle

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('data/bbc_news.csv', encoding='utf-8')
df.head(10)

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...
5,Ukraine war: PM to hold talks with world leade...,"Mon, 07 Mar 2022 08:33:29 GMT",https://www.bbc.co.uk/news/uk-60642926,https://www.bbc.co.uk/news/uk-60642926?at_medi...,Boris Johnson is to meet the Canadian and Dutc...
6,Ukraine war: UK grants 50 Ukrainian refugee vi...,"Mon, 07 Mar 2022 08:09:21 GMT",https://www.bbc.co.uk/news/uk-60640460,https://www.bbc.co.uk/news/uk-60640460?at_medi...,"The home secretary says she is ""surging capaci..."
7,TikTok limits services as Netflix pulls out of...,"Mon, 07 Mar 2022 00:11:59 GMT",https://www.bbc.co.uk/news/business-60641988,https://www.bbc.co.uk/news/business-60641988?a...,TikTok suspends live streaming and new content...
8,"Covid: Fourth jab for Scotland's vulnerable, a...","Mon, 07 Mar 2022 07:46:30 GMT",https://www.bbc.co.uk/news/uk-60640975,https://www.bbc.co.uk/news/uk-60640975?at_medi...,Five things you need to know about the coronav...
9,Protests across Russia see thousands detained,"Sun, 06 Mar 2022 23:23:59 GMT",https://www.bbc.co.uk/news/world-europe-60640204,https://www.bbc.co.uk/news/world-europe-606402...,"People have been held in 53 cities, from St Pe..."


In [ ]:
from src.iswd723 import clean_text, stemming, remove_stopwords, filter_tokens
df['clean_text'] = df['description'].apply(clean_text).apply(remove_stopwords).apply(filter_tokens).apply(stemming)
df['clean_text'].head(10)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,clean_text
0,ukrainian presid say countri forgiv forget mur...
1,jeremi bowen frontlin irpin resid came russian...
2,one world biggest fertilis firm say conflict c...
3,parent manchest arena bomb youngest victim spe...
4,consum feel impact higher energi cost fuel pri...
5,bori johnson meet canadian dutch debat new law...
6,home secretari say surg capac applic grant
7,tiktok suspend live stream new content platfor...
8,five thing need know coronaviru pandem monday ...
9,peopl held citi petersburg west vladivostok east


In [ ]:
query = "The war between Syria and Iraq leaves thousands wounded"

### Creación de indice invertido

In [ ]:
from src.iswd723 import make_index_inv
indice_inv = make_index_inv(df['clean_text'])
indice_inv_tf = {i : {'docs':indice_inv[i], 'freq':len(indice_inv[i])} for i in indice_inv.keys()}

### Modelo binario

In [ ]:
from src.iswd723 import build_vocabulary
vocab = build_vocabulary(df['clean_text'])
vocab_map = {word : i for i, word in enumerate(vocab)}

In [ ]:
bin_vector = np.zeros((len(df['clean_text']),len(vocab)), dtype=int) #mat se almacena el vector binario
for key, docs in indice_inv.items():
    bin_vector[docs, [vocab_map[key]]] = 1

In [ ]:
query_binary_model = stemming(remove_stopwords(clean_text(query)))
q_bin_vector = np.zeros(len(vocab), dtype = int)
for word in query_binary_model.split():
    if vocab_map.get(word, -1) != -1:
        q_bin_vector[vocab_map[word]] = 1 #vector binario para la query
    else:
        continue

In [ ]:
from src.iswd723 import jaccard_similarity
sim_bin = [jaccard_similarity(vect, q_bin_vector) for vect in bin_vector]

In [ ]:
mat_sim_bin = pd.DataFrame(
    {
        'title':df['title'],
        'Description':df['description'],
        'URL': df['link'],
        'sim_bin':sim_bin
    }
)
mat_sim_bin.sort_values(by='sim_bin', ascending=False).head(10)

,title,Description,URL,sim_bin
14471,Ukraine war: The cost and scale of rehabilitat...,Ukraine is trying to rehabilitate thousands of...,https://www.bbc.co.uk/news/world-europe-648629...,0.230769
269,Ukraine war: Indian students on the trauma of ...,The BBC follows some of the thousands of stude...,https://www.bbc.co.uk/news/world-asia-60689267...,0.222222
13602,Turkey-Syria earthquake: King thanked for 'gen...,"More than 20,000 people are dead and thousands...",https://www.bbc.co.uk/news/uk-64568254?at_medi...,0.200000
14368,Exodus from Cuba: The children left behind as ...,Hundreds of thousands are emigrating with many...,https://www.bbc.co.uk/news/world-latin-america...,0.166667
33135,Israel orders more evacuations as Rafah fighti...,The Israeli military tells thousands of Palest...,https://www.bbc.com/news/articles/cer3mr09vzxo,0.153846
14959,What started the 2003 Iraq war?,The US and three allies invaded Iraq 20 years ...,https://www.bbc.co.uk/news/world-64980565?at_m...,0.153846
5817,Ukraine conflict: How are are drones being used?,Thousands of military and civilian drones are ...,https://www.bbc.co.uk/news/world-62225830?at_m...,0.153846
39384,'We just had to flee': Fear and tension in Leb...,Thousands have fled southern Lebanon while oth...,https://www.bbc.com/news/articles/crl8xw7ww70o,0.153846
9328,Ethiopia civil war: Hyenas scavenge on corpses...,Horrific accounts from a brutal war in Ethiopi...,https://www.bbc.co.uk/news/world-africa-632917...,0.153846
13711,Turkey earthquake: Where did it hit and why wa...,Two earthquakes have killed thousands of peopl...,https://www.bbc.co.uk/news/science-environment...,0.153846


### Modelo TFIDF

In [ ]:
from src.iswd723 import get_tf, get_df, get_idf, get_tfidf

tf = df['clean_text'].apply(lambda x: get_tf(x, vocab_map, len(vocab))) #Diccionario TF -> N. Doc : vector TF
tf_ = np.stack(tf.values) #Matriz tf : np.array
df_ = get_df(tf_)
idf = get_idf(df['clean_text'], df_)
tfidf__ = get_tfidf(tf_, idf)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
tf_idf = vect.fit_transform(df['clean_text'])
mat_tfidf = pd.DataFrame(
    tf_idf.toarray(),
    columns = vect.get_feature_names_out()
)
mat_tfidf.head()

,aa,aaliyah,aamir,aanoch,aardman,aaron,aaronovitch,aastha,ab,abandon,...,zuniga,zunino,zunyou,zurcher,zurich,zuu,zuwara,zverev,zwelithini,zwerner
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from src.iswd723 import calculate_cos_similarity
q_tfidf = vect.transform([query_binary_model])
cos_sims = calculate_cos_similarity(tfidf__, q_tfidf.toarray())
ranking_idx_tfidf = np.argsort(-cos_sims)

In [ ]:
ranking_df_tfidf = pd.DataFrame({
    'idx': df.index[ranking_idx_tfidf],
    'document': df['description'].iloc[ranking_idx_tfidf],
    'URL':df['link'].iloc[ranking_idx_tfidf],
    'tfidf_similarity': cos_sims[ranking_idx_tfidf]
})

ranking_df_tfidf.head(10)

,idx,document,URL,tfidf_similarity
28760,28760,The strikes in Syria and Iraq will take place ...,https://www.bbc.co.uk/news/world-us-canada-681...,0.421927
28123,28123,Islamabad says the attack killed two children ...,https://www.bbc.co.uk/news/world-asia-67999465...,0.363135
14471,14471,Ukraine is trying to rehabilitate thousands of...,https://www.bbc.co.uk/news/world-europe-648629...,0.350077
28922,28922,President Joe Biden says he will order new act...,https://www.bbc.co.uk/news/world-us-canada-682...,0.343895
28880,28880,BBC's Paul Adams asks whether US strikes on Ir...,https://www.bbc.co.uk/news/world-middle-east-6...,0.339471
28865,28865,The Pentagon has struck Iran-linked targets in...,https://www.bbc.co.uk/news/world-us-canada-681...,0.339397
28948,28948,The Pentagon has struck Iran-linked targets in...,https://www.bbc.co.uk/news/world-us-canada-681...,0.339397
13602,13602,"More than 20,000 people are dead and thousands...",https://www.bbc.co.uk/news/uk-64568254?at_medi...,0.339221
14959,14959,The US and three allies invaded Iraq 20 years ...,https://www.bbc.co.uk/news/world-64980565?at_m...,0.329720
13711,13711,Two earthquakes have killed thousands of peopl...,https://www.bbc.co.uk/news/science-environment...,0.299235


In [ ]:
# Tokenizar los documentos (lista de listas de tokens)
docs_tokenized = df['clean_text'].apply(lambda x: x.split())

In [ ]:
# 1. Calcular la longitud promedio de los documentos (avg_dl)
from src.iswd723 import avgdl
avg_dl = avgdl(docs_tokenized)
print(f"Longitud promedio de documentos (avg_dl): {avg_dl:.2f}")

Longitud promedio de documentos (avg_dl): 10.68


In [ ]:
# 2. Construir el índice básico para TF crudo y DF (document frequency)
from src.iswd723 import basic_index
tf_raw, df_term = basic_index(docs_tokenized)

In [ ]:
# 3. Calcular IDF con el modelo RSJ (Robertson-Sparck Jones)
from src.iswd723 import idf_rsj
N = len(docs_tokenized)
idf_bm25 = idf_rsj(df_term, N)

In [ ]:
# 4. Calcular la matriz de scores BM25
from src.iswd723 import calculate_scores
scores_bm25, terms_bm25 = calculate_scores(docs_tokenized, avg_dl, tf_raw, idf_bm25)

In [ ]:
# 5. Aplicar el ranking BM25 a la consulta
from src.iswd723 import bm25_rank_query

# query_binary_model = 'war syria iraq leav thousand wound' (Ejemplo del preprocesamiento)
ranking_idx_bm25, sim_bm25 = bm25_rank_query(query_binary_model, terms_bm25, scores_bm25)

In [ ]:
# 6. Mostrar el ranking
ranking_df_bm25 = pd.DataFrame({
    'idx': df.index[ranking_idx_bm25],
    'document': df['description'].iloc[ranking_idx_bm25],
    'URL': df['link'].iloc[ranking_idx_bm25],
    'bm25_similarity': sim_bm25
})

print("Top 10 resultados con BM25:")
ranking_df_bm25.head(10)

Top 10 resultados con BM25:


,idx,document,URL,bm25_similarity
14471,14471,Ukraine is trying to rehabilitate thousands of...,https://www.bbc.co.uk/news/world-europe-648629...,1.000000
13602,13602,"More than 20,000 people are dead and thousands...",https://www.bbc.co.uk/news/uk-64568254?at_medi...,0.887586
28760,28760,The strikes in Syria and Iraq will take place ...,https://www.bbc.co.uk/news/world-us-canada-681...,0.885220
28865,28865,The Pentagon has struck Iran-linked targets in...,https://www.bbc.co.uk/news/world-us-canada-681...,0.849881
28880,28880,BBC's Paul Adams asks whether US strikes on Ir...,https://www.bbc.co.uk/news/world-middle-east-6...,0.849881
28123,28123,Islamabad says the attack killed two children ...,https://www.bbc.co.uk/news/world-asia-67999465...,0.849881
28948,28948,The Pentagon has struck Iran-linked targets in...,https://www.bbc.co.uk/news/world-us-canada-681...,0.849881
14959,14959,The US and three allies invaded Iraq 20 years ...,https://www.bbc.co.uk/news/world-64980565?at_m...,0.793558
28922,28922,President Joe Biden says he will order new act...,https://www.bbc.co.uk/news/world-us-canada-682...,0.787043
13711,13711,Two earthquakes have killed thousands of peopl...,https://www.bbc.co.uk/news/science-environment...,0.766824
